
# Day 1b: Agent Architectures（LangChain可本地运行版）

## 🚀 第一天（b）- 代理架构

欢迎来到 5 天 Agent 课程的 Day 1b！

在 Day 1a 中，你构建了一个能够使用工具的单一 Agent。现在，我们将升级到**多 Agent 系统**。

就像现实世界中的团队一样，你可以创建一组**专业化**的 Agent，它们协作解决复杂问题。这是 AI Agent 开发中最强大的概念之一。

在本 Notebook 中，你将：

- ✅ **了解何时使用多 Agent 系统**
- ✅ **构建你的第一个多 Agent 系统** (使用 LLM 作为"经理")
- ✅ **掌握三种核心工作流模式**：
    1. **顺序模式 (Sequential)**: 像流水线一样一步步执行
    2. **并行模式 (Parallel)**: 同时执行多个任务以提高效率
    3. **循环模式 (Loop)**: 通过反复修正来提高质量

---

**原版**: Google ADK Team (Kaggle 5-Day Agent Course)  
**改写**: LangChain 实现
> **ℹ️ 声明**: 本教程改编自 Google [5-Day AI Agents Course](https://www.kaggle.com/learn-guide/5-day-agents)，由 LangChain 重写以适配本地运行。内容仅供学习研究。



## ⚙️ Section 1: Setup (环境设置)

### 1.1 切换内核
- 点击右上角的 Kernel (内核) 选择区域。
- 在列表中选择 Python (5-DAY-AGENTS)。

> 关于如何初始化环境和创建内核，请查看 [README 如何运行](../README.md#-如何运行)

### 1.2 安装依赖

In [ ]:
# 优选 uv 包管理器 
!uv pip install langchain langchain-openai langchain-community python-dotenv duckduckgo-search tavily-python


Using Python 3.11.11 environment at: /Users/jupiter/code/5-DAY-AI-AGENT/.venv
Audited 6 packages in 32ms


### 1.3 导入依赖库

In [3]:

import os
import json
import warnings
from dotenv import load_dotenv, find_dotenv

# LangChain 核心组件
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.agents import AgentExecutor, create_tool_calling_agent

# 搜索工具
from langchain_community.tools import DuckDuckGoSearchResults, TavilySearchResults

# 忽略警告
from langchain_core._api.deprecation import LangChainDeprecationWarning
warnings.simplefilter("ignore", category=LangChainDeprecationWarning)

print("✅ 依赖库导入成功")

✅ 依赖库导入成功


### 1.4 配置 API Key

> 关于 .env 文件内容说明，请查看 [README 环境配置文件 (.env) 说明](../README.md#3-环境配置文件-env-说明)

In [4]:
# 加载环境变量
load_dotenv(find_dotenv())

def configure_api_keys():
    """加载并验证 API 配置"""
    model_name = os.environ.get("MODEL_NAME")
    api_key = os.environ.get("OPENAI_API_KEY")
    base_url = os.environ.get("OPENAI_API_BASE")
    tavily_key = os.environ.get("TAVILY_API_KEY")

    if not api_key:
        print("❌ 错误: 未在 .env 文件中找到 OPENAI_API_KEY")
        return None
    
    # 打印部分 Key 以验证 (隐藏敏感信息)
    masked_key = f"{api_key[:8]}..." if api_key else "None"
    print(f"✅ API Key 已加载: {masked_key}")
    print(f"✅ 使用模型: {model_name}")
    if base_url:
        print(f"✅ API Base URL: {base_url}")
    
    if tavily_key:
        print("✅ Tavily Search API 已启用 (增强搜索能力)")
    else:
        print("⚠️ 未找到 Tavily Key，将使用 DuckDuckGo 搜索 (免费但质量可能受限)")
    
    return {
        "model_name": model_name,
        "api_key": api_key,
        "base_url": base_url,
        "tavily_key": tavily_key,
        "extra_body": json.loads(os.environ.get("EXTRA_BODY", "{}"))
    }

config = configure_api_keys()

✅ API Key 已加载: ms-04d57...
✅ 使用模型: Qwen/Qwen3-235B-A22B-Instruct-2507
✅ API Base URL: https://api-inference.modelscope.cn/v1
✅ Tavily Search API 已启用 (增强搜索能力)


### 1.5 初始化 LLM

现在，让我们初始化 LangChain 的 ChatOpenAI 模型。这相当于 ADK 中的 `Gemini` 模型配置。

In [5]:
# 初始化 LLM
llm = ChatOpenAI(
    model=config["model_name"],
    api_key=config["api_key"],
    base_url=config["base_url"],
    temperature=0.7,
    extra_body=config["extra_body"]
)

print("✅ LLM 初始化完成")

✅ LLM 初始化完成



### 1.6 定义通用工具

我们将继续使用 Day 1a 中的网络搜索工具。


In [6]:

@tool
def internet_search(query: str) -> str:
    '''
    Performs an internet search using Tavily (preferred) or DuckDuckGo (fallback).
    Use this tool to find answer about current events, weather, or specific facts.
    '''
    if config["tavily_key"]:
        try:
            # print(f"    🔍 [Search] Tavily: '{query}'")
            return TavilySearchResults(k=3).run(query)
        except Exception:
            pass
    
    # print(f"    🔍 [Search] DuckDuckGo: '{query}'")
    return DuckDuckGoSearchResults(backend="api").run(query)

print("✅ 工具 internet_search 已准备就绪")


✅ 工具 internet_search 已准备就绪



## 🤔 Section 2: 为什么要使用多 Agent 系统？

**问题：** 单个 "全能" Agent 往往会顾此失彼。如果让一个 Agent 同时负责研究、写作、编辑和核查，它的 Prompt 会变得非常长且难以维护，且容易出错。

**解决方案：** **专家团队**。
我们将任务分解给多个角色单一的 Agent（例如，一个只负责搜索，另一个只负责总结）。

### 2.1 示例：研究与总结系统 (Coordinator 模式)

我们构建一个系统，包含：
1. **研究员 (Research Agent)**: 使用搜索工具查找信息。
2. **总结员 (Summarizer Agent)**: 将查找的信息整理成摘要。
3. **协调员 (Coordinator/Root Agent)**: 负责指挥前两个 Agent 工作。

在 LangChain 中，我们可以将 "子 Agent" 封装为 **Tool** 供 "根 Agent" 调用。


In [7]:

# --- 1. 定义 研究员 (Research Agent) ---
# 它是一个可以说 "查找关于 X 的信息" 的 Agent
research_prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个专业的研究员。你的任务是使用搜索工具查找关于用户给定主题的详细信息。请查找 2-3 个关键点并提供来源。"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])
research_agent = create_tool_calling_agent(llm, [internet_search], research_prompt)
research_executor = AgentExecutor(agent=research_agent, tools=[internet_search], verbose=False)

# 将其封装为工具供 Coordinator 使用
@tool
def research_tool(topic: str) -> str:
    '''用于研究特定主题的工具。输入应该是从用户问题中提取的研究主题。'''
    print(f"  🤖 [ResearchAgent] 正在研究: {topic}...")
    result = research_executor.invoke({"input": topic})
    return result["output"]

print("✅ research_agent created.")

✅ research_agent created.


In [8]:
# --- 2. 定义 总结员 (Summarizer Agent) ---
# 它只需要处理文本，不需要外部工具，所以我们可以用简单的 Chain
summarizer_prompt = ChatPromptTemplate.from_template(
    "阅读以下研究资料:\n\n{research_findings}\n\n请将其总结为一个包含 3-5 个要点的列表，语言简洁明了。"
)
# 使用 LCEL: Prompt -> LLM -> OutputParser
summarizer_chain = summarizer_prompt | llm | StrOutputParser()

@tool
def summarizer_tool(content: str) -> str:
    '''用于总结文本的工具。输入应该是需要总结的长文本。'''
    print(f"  📝 [SummarizerAgent] 正在总结...")
    return summarizer_chain.invoke({"research_findings": content})

print("✅ summarizer_agent created.")

✅ summarizer_agent created.


In [9]:
# --- 3. 定义 协调员 (Root/Coordinator Agent) ---
coordinator_prompt = ChatPromptTemplate.from_messages([
    ("system", '''你是一名研究协调员。你的目标是通过通过协调子 Agent 来回答用户的问题。
    
    请遵循以下步骤：
    1. 调用 `research_tool` 查找相关信息。
    2. 获取研究结果后，调用 `summarizer_tool` 进行总结。
    3. 最后，将总结结果展示给用户。'''),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

root_tools = [research_tool, summarizer_tool]
coordinator_agent = create_tool_calling_agent(llm, root_tools, coordinator_prompt)
coordinator_runner = AgentExecutor(agent=coordinator_agent, tools=root_tools, verbose=True, handle_parsing_errors=True) # Verbose True 以便看到调用过程

print("✅ 多 Agent 系统 (Coordinator 模式) 创建完成")

✅ 多 Agent 系统 (Coordinator 模式) 创建完成



### 运行 Coordinator Agent


In [10]:

query = "量子计算的最新进展及其对 AI 的意义"
print(f"User > {query}\n")

response = coordinator_runner.invoke({"input": query})
print(f"\nRootAgent > {response['output']}")


User > 量子计算的最新进展及其对 AI 的意义



> Entering new AgentExecutor chain...

Invoking: `research_tool` with `{'topic': '量子计算的最新进展及其对 AI 的意义'}`


  🤖 [ResearchAgent] 正在研究: 量子计算的最新进展及其对 AI 的意义...
以下是关于量子计算最新进展及其对人工智能（AI）意义的两个关键点，并附有信息来源：

---

**1. 量子计算硬件取得重大突破，模块化系统推动实用性发展**  
IBM于2023年12月推出“量子系统二号”（Quantum System Two），这是其首款可模块化扩展的量子计算机，拥有超过1000个量子位。该系统的模块化设计允许未来连接多个处理器，实现算力叠加，标志着量子计算机正从实验室走向实用化。IBM还设定了十年内建成10万量子位系统的长期目标，为实现通用量子计算奠定基础。

> 来源：新华网、中国新闻网  
> - [http://www.news.cn/tech/20240103/...](http://www.news.cn/tech/20240103/06334b17b41c44518168c2dea7bb844d/c.html)  
> - [http://www.xinhuanet.com/info/20240104/...](http://www.xinhuanet.com/info/20240104/12f414e338084cf8b52e126e2445e330/c.html)

---

**2. 量子计算与AI融合加速，AI助力量子纠错，量子提升AI算力瓶颈**  
近年来，“量子机器学习”成为研究热点。一方面，AI被用于解决量子计算的核心难题——量子纠错。例如，谷歌利用基于神经网络的解码器，通过机器学习显著降低量子操作中的错误率，相当于为量子计算机安装“智能纠错系统”。另一方面，量子计算因其天然并行性，在处理复杂AI任务（如优化、主成分分析）时具备理论优势，有望缓解当前大模型（如ChatGPT）面临的算力瓶颈。

> 来源：上海日报、CCF官网  
> - [https://m.jfdaily.com/wx/detail.do?id=986775](https://m


## 🚥 Section 3: 顺序工作流 (Sequential Workflows) - 流水线

**问题：** 仅仅依靠 LLM "决定" 调用谁（如上面的 Coordinator）有时会不稳定。如果 LLM 决定跳过一步怎么办？

**解决方案：** **固定流水线 (Sequential/Chain)**。
当任务顺序固定（A -> B -> C）时，我们可以强制定义执行顺序。

### 3.1 示例：博客创作流水线

我们将构建一个包含三个环节的流水线：
1. **大纲 (Outline)**: 生成博客大纲
2. **写作 (Write)**: 根据大纲写草稿
3. **编辑 (Edit)**: 润色草稿

在 LangChain 中，这是最经典的 **LCEL Chain** (`|` 运算符)。


In [11]:

# 1. 大纲生成器
outline_prompt = ChatPromptTemplate.from_template(
    "既然你是 OutlineAgent。请为主题 '{topic}' 创建一个博客大纲，包含引人入胜的标题、介绍、3-5个主要段落要点和结论。"
)
outline_chain = outline_prompt | llm | StrOutputParser()

# 2. 撰写工
writer_prompt = ChatPromptTemplate.from_template(
    "既然你是 WriterAgent。请根据以下大纲撰写一篇 200-300 字的博客文章，语气要生动有趣。\n\n大纲:\n{outline}"
)
writer_chain = writer_prompt | llm | StrOutputParser()

# 3. 编辑
editor_prompt = ChatPromptTemplate.from_template(
    "既然你是 EditorAgent。请编辑以下草稿，修正语法错误，优化流畅度，并使其更加清晰。\n\n草稿:\n{draft}"
)
editor_chain = editor_prompt | llm | StrOutputParser()

# --- 构建顺序链 (Sequential Chain) ---
# 逻辑: Unput(Topic) -> Outline -> Writer -> Editor -> Final Output
# 我们使用 RunnablePassthrough 来传递中间变量

from langchain_core.runnables import RunnablePassthrough

# 定义完整的流水线
# 步骤 1: 生成大纲
chain_step1 = {"outline": outline_chain} 

# 步骤 2: 生成草稿 (输入是上一步的字典，包含 outline)
chain_step2 = {"draft": writer_chain}

# 步骤 3: 编辑草稿 (输入是上一步的字典，包含 draft)
chain_step3 = editor_chain

# 连接起来
# 注意: 为了让每一步都能拿到它需要的输入变量，我们需要巧妙地组合 Runnables
# 更简单的方法是显式地传递数据流

def run_blog_pipeline(topic):
    print(f"📝 [OutlineAgent] 正在生成大纲: '{topic}'...")
    outline = outline_chain.invoke({"topic": topic})
    print(f"\n--- 大纲 ---\n{outline[:100]}...\n")
    
    print(f"✍️ [WriterAgent] 正在撰写草稿...")
    draft = writer_chain.invoke({"outline": outline})
    print(f"\n--- 初稿 ---\n{draft[:100]}...\n")
    
    print(f"🔍 [EditorAgent] 正在润色...")
    final_post = editor_chain.invoke({"draft": draft})
    
    return final_post

print("✅ 博客创作流水线已就绪 (Function-based sequential workflow)")


✅ 博客创作流水线已就绪 (Function-based sequential workflow)


In [12]:

# 运行流水线
topic = "为软件开发人员介绍多 Agent 系统的优势"
final_article = run_blog_pipeline(topic)

print(f"\n🚀 [Final Output] 最终成文:\n\n{final_article}")


📝 [OutlineAgent] 正在生成大纲: '为软件开发人员介绍多 Agent 系统的优势'...

--- 大纲 ---
**博客大纲：为软件开发人员介绍多 Agent 系统的优势**

---

**引人入胜的标题：**  
《从单打独斗到协同作战：为什么多 Agent 系统是软件开发者的下一个利器？》

---

*...

✍️ [WriterAgent] 正在撰写草稿...

--- 初稿 ---
**《从单打独斗到协同作战：为什么多 Agent 系统是软件开发者的下一个利器？》**

还在让一个服务“孤军奋战”扛下所有逻辑？醒醒，程序员的未来不是“单机模式”，而是“组队开黑”！欢迎来到**多 ...

🔍 [EditorAgent] 正在润色...

🚀 [Final Output] 最终成文:

当然可以！以下是经过语法修正、语言优化和逻辑梳理后的编辑版本，在保持原文生动风格的基础上，提升了专业性、流畅度与表达清晰度：

---

# **从单打独斗到协同作战：为什么多 Agent 系统是软件开发者的下一个利器？**

还在让一个服务“孤军奋战”扛下所有业务逻辑？是时候告别“单机模式”了——程序员的未来，早已转向“组队开黑”的智能协作时代。欢迎进入**多 Agent 系统（Multi-Agent System, MAS）**的世界。

在这里，每个 **Agent** 都是一个具备自治性、目标导向和环境感知能力的智能体，就像一支无需团建也能默契配合的“数字梦之队”。它们能独立思考、主动响应，更擅长彼此协作，共同完成复杂任务。

### 什么是多 Agent 系统？

简单来说，Agent 是能够感知环境、做出决策并采取行动的小型智能单元。当多个 Agent 组成系统，便能通过分工与协作，解决单一组件难以应对的挑战。  
比如：自动驾驶车辆间的避障协商、运维系统中自动扩缩容的“机器人小分队”，或是供应链网络中的动态资源调配——这些场景背后，都是多 Agent 协同在发挥作用。

相比传统的微服务架构只强调“拆分”，MAS 更进一步：它不仅解耦功能，更注重**智能协作**与**动态协调**。

---

### 多 Agent 系统的三大核心优势

#### 1. **超强扩展性：灵活增减，无缝集成**
新 Agent 可随时加入，旧模块也可平滑退役


## 🛣️ Section 4: 并行工作流 (Parallel Workflows)

**问题：** 顺序执行虽然稳定，但如果任务之间没有依赖关系（例如研究三个不同的独立主题），顺序执行太慢了。

**解决方案：** **并行执行 (Parallel/Concurrent)**。
同时运行多个 Agent，最后聚合结果。

### 4.1 示例：多主题研究简报

我们将创建三个研究员并行工作，然后由一个聚合员汇总：
1. **Tech Researcher**: 研究 AI 趋势
2. **Health Researcher**: 研究医疗突破
3. **Finance Researcher**: 研究金融科技
4. **Aggregator**: 汇总简报

在 LangChain 中，我们使用 `RunnableParallel` 来实现并行。


In [13]:

# 定义通用的研究 Chain (复用之前的 research agent 逻辑会更复杂，这里我们用简单的 Chain + Tool)
# 为了演示并行，我们直接创建一个能够通过 Search Tool 查找信息的 Chain

def create_research_chain(role_name, focus_area):
    prompt = ChatPromptTemplate.from_messages([
        ("system", f"你是 {role_name}。请研究 '{focus_area}' 的最新动态。找出 3 个关键发展。保持简炼(100字以内)。"),
        ("human", "{input}"), # input 主要是触发 trigger，或者可以为空
        ("placeholder", "{agent_scratchpad}"),
    ])
    # 这里我们复用之前带有 search tool 的 agent 结构，但为了并行运行方便，封成一个 Runnable
    agent = create_tool_calling_agent(llm, [internet_search], prompt)
    executor = AgentExecutor(agent=agent, tools=[internet_search], verbose=False)
    return executor

# 创建三个并行 Agent
tech_agent = create_research_chain("TechResearcher", "AI and Machine Learning Trends")
health_agent = create_research_chain("HealthResearcher", "Medical Breakthroughs and Health News")
finance_agent = create_research_chain("FinanceResearcher", "Fintech and Economy Trends")

# 聚合 Agent
aggregator_prompt = ChatPromptTemplate.from_template(
    '''将以下三个研究报告汇总成一份执行简报：

    **Technology:**
    {tech_output}

    **Health:**
    {health_output}

    **Finance:**
    {finance_output}

    请提取共同主题和关键要点。'''
)
aggregator_chain = aggregator_prompt | llm | StrOutputParser()

# 构建并行组件
# RunnableParallel 会同时通过 invoke 调用其中的每个 runnable
parallel_research = RunnableParallel({
    "tech_output": lambda x: tech_agent.invoke({"input": "research latest trends"})['output'],
    "health_output": lambda x: health_agent.invoke({"input": "research latest trends"})['output'],
    "finance_output": lambda x: finance_agent.invoke({"input": "research latest trends"})['output'],
})

# 构建完整流程
full_parallel_chain = parallel_research | aggregator_chain

print("✅ 并行研究工作流已构建")


✅ 并行研究工作流已构建


In [ ]:

print("🚀 开始并行研究 (这可能需要一分钟，因为要进行多次搜索)...")
final_briefing = full_parallel_chain.invoke("Start") 
print(f"\n📊 [Executive Briefing] 每日简报:\n\n{final_briefing}")


🚀 开始并行研究 (这可能需要一分钟，因为要进行多次搜索)...


RateLimitError: Error code: 429 - {'errors': {'message': 'We have to rate limit you for model Qwen/Qwen3-235B-A22B-Instruct-2507. If you need higher limits, please consider other (commercial) API providers.', 'request_id': '7859e08a-a338-458b-849c-2d619420e9ed'}}


## ➰ Section 5: 循环工作流 (Loop Workflows) - 反复打磨

**问题：** 之前的模式都是"一次性"的 (One-Shot)。但很多高质量的工作需要"写-评-改"的循环。

**解决方案：** **循环优化 (Iterative Refinement)**。
设置一个循环：生成 -> 评估 -> (如果不合格) -> 改进 -> 再次评估...

### 5.1 示例：故事创作与批评循环

1. **Writer**: 写初稿
2. **Critic**: 提出修改意见，或者说 "APPROVED"
3. **Refiner**: 根据意见修改文章

这个逻辑通常使用 Python 的 `while` 循环或者 `LangGraph` 来实现。这里我们用 Python 原生循环演示这一逻辑。


In [15]:

# 1. 初始作家
writer_prompt = ChatPromptTemplate.from_template(
    "请根据主题 '{topic}' 写一个短篇故事（100-150字）。只输出故事内容。"
)
initial_writer = writer_prompt | llm | StrOutputParser()

# 2. 批评家
critic_prompt = ChatPromptTemplate.from_template(
    '''你是一个严厉的批评家。请审阅下面的故事：
    
    Story: {story}
    
    如果不完美，请给出 2-3 条具体的修改建议。
    如果故事非常出色且无需修改，请只回复: APPROVED'''
)
critic_agent = critic_prompt | llm | StrOutputParser()

# 3. 润色家
refiner_prompt = ChatPromptTemplate.from_template(
    '''你是一个编辑。请根据批评意见重写故事。
    
    原故事: {story}
    批评意见: {critique}
    
    请输出修改后的完整故事。'''
)
refiner_agent = refiner_prompt | llm | StrOutputParser()

# --- 循环逻辑函数 ---
def run_refinement_loop(topic, max_iterations=3):
    print(f"✍️ [Writer] 正在写初稿: {topic}...")
    current_story = initial_writer.invoke({"topic": topic})
    print(f"\n📜 [Draft 0]:\n{current_story}\n")
    
    for i in range(max_iterations):
        print(f"--- Iteration {i+1} ---")
        
        # 批评
        print(f"🧐 [Critic] 正在审阅...")
        critique = critic_agent.invoke({"story": current_story})
        print(f"  > 意见: {critique}")
        
        if "APPROVED" in critique:
            print("\n✅ 故事已获得批准！")
            break
            
        # 润色
        print(f"🔧 [Refiner] 正在根据意见修改...")
        current_story = refiner_agent.invoke({"story": current_story, "critique": critique})
        print(f"\n📜 [Draft {i+1}]:\n{current_story}\n")
        
    return current_story

print("✅ 循环优化工作流已定义")


✅ 循环优化工作流已定义


In [21]:

topic = "一个灯塔看守人发现了一张发光的神秘地图"
final_story = run_refinement_loop(topic)

print(f"\n🏆 [Final Story]:\n{final_story}")


✍️ [Writer] 正在写初稿: 一个灯塔看守人发现了一张发光的神秘地图...

📜 [Draft 0]:
暴风雨夜，灯塔看守人老陈擦拭透镜时，发现墙缝夹着一张泛黄地图。他取下拂尘，地图竟幽幽亮起，蓝光勾勒出从未见过的海底峡谷与沉船轮廓。更奇的是，灯塔日志里无端多出一行字：“第七个满月，潮退之时，门将开启。”老陈望向窗外翻涌的黑海，忽然明白——百年来每任看守失踪的真相，皆因这张地图的召唤。今夜，轮到他了。

--- Iteration 1 ---
🧐 [Critic] 正在审阅...
  > 意见: 1. 故事氛围营造出色，但关键信息揭示略显仓促。建议延展老陈发现日志异变的过程，增加他翻阅前人记录、发现失踪规律的细节，以增强悬疑的累积感和宿命的压迫感。

2. “地图幽幽亮起”这一超自然现象缺乏铺垫。建议在开篇加入环境异象（如灯塔光束异常闪烁、海浪声诡异地同步心跳），使后续奇幻元素的出现更自然，提升整体神秘质感。

3. 结尾“今夜，轮到他了”虽有力，但动机模糊。应补充老陈内心的挣扎或使命感来源（如他曾发誓守护灯塔秘密，或地图映出他失踪父亲的遗物），以深化人物抉择的悲剧性与必然性。
🔧 [Refiner] 正在根据意见修改...

📜 [Draft 1]:
**修改后的故事：**

暴风雨在午夜时分骤然降临。灯塔的光束本该稳定地扫过海面，此刻却忽明忽暗，像一只疲倦的眼睛，在浓墨般的浪涛中艰难睁开又闭合。老陈皱眉抬头，手中的鹿皮布停在透镜边缘。他在这座孤悬海外的灯塔守了三十七年，从未见过光束如此紊乱——更奇怪的是，那闪烁的节奏，竟与他胸腔里的心跳隐隐同步。

窗外，黑海翻腾如沸，浪头拍打着礁石，发出沉闷的轰响，可那声音……似乎也带着某种规律，像是低语，又像召唤。

老陈放下布巾，走向控制台检查电路。指尖刚触到开关，余光却瞥见墙缝间有异样——一张泛黄的纸片，被岁月压得几乎与石墙融为一体，只露出一角卷曲的边。他愣住。这缝隙他每年粉刷都亲手填补，从不可能藏匿东西。

他用螺丝刀小心撬开砖缝，取出那张薄纸。地图。不是现代测绘的样式，而是手绘的墨线，笔触苍老而精准，描绘着一片从未出现在任何海图上的海域：一道深不见底的海底峡谷蜿蜒如龙脊，峡谷尽头，一艘巨船倒扣于岩床之上，船身缠绕着珊瑚与铁锈般的阴影。更诡异的是，当他将地图举至灯下，它忽然幽幽亮起，泛出淡蓝微光，仿佛海水本身在


## ✅ 总结与回顾

恭喜！你刚刚构建了三种强大的多 Agent 设计模式：

| 模式 | 适用场景 | 示例 | 关键特征 |
| :--- | :--- | :--- | :--- |
| **Coordinator (LLM决定)** | 需要动态决策 | 研究协调员 | LLM 决定调用哪个 Agent/Tool |
| **Sequential (流水线)** | 顺序固定的任务 | 博客创作 (大纲->写->改) | 确定性的一步接一步 |
| **Parallel (并行)** | 相互独立的任务 | 多主题研究 | 同时运行，速度快 |
| **Loop (循环)** | 需要质量打磨 | 写作+批评循环 | 反复迭代直到满意 |

这些模式是构建复杂 AI 应用的基石。在接下来的课程中，我们将深入探讨如何让这些 Agent 拥有更长期的记忆和更复杂的工具使用能力。
